In [193]:
# !pip install tweepy

In [1]:

import tweepy
CONSUMER_KEY = ''
CONSUMER_SECRET = ''
ACCESS_TOKEN = ''
ACCESS_SECRET  = ''
BEARER_TOKEN = ''
# Setup tweepy to authenticate with Twitter credentials:


# Create the api to connect to twitter with your credentials.  Note that these are my credentials; you'll have to use your own

client = tweepy.Client(
    bearer_token=BEARER_TOKEN,
    consumer_key=CONSUMER_KEY,
    consumer_secret=CONSUMER_SECRET,
    access_token=ACCESS_TOKEN,
    access_token_secret=ACCESS_SECRET
)

In [80]:
geo_data_extended = []

In [3]:
data_path = "/Users/nihaldsouza/code/covid_hate/data/asonam_release_all_tweets.csv"

In [147]:
import pandas as pd

data_pd = pd.read_csv(data_path, skiprows=range(1,3003100), nrows=1000000)
data_pd.head(5)

,Tweet ID,BERT_label
0,1222268238354178050,0
1,1222275145332473856,0
2,1222277574111436801,0
3,1222273373461536769,0
4,1222263502578036737,0


In [148]:
hate_tweets_pd = data_pd[data_pd['BERT_label'] == 2]
hate_tweets_pd.shape

(17021, 2)

In [149]:
import time
from tqdm.auto import tqdm

def get_location(tweet_id_list, label_list):
    header = {"Authorization": "Bearer "+BEARER_TOKEN}
    geo_data = []
    for i, tweet_id in tqdm(enumerate(tweet_id_list), total=len(tweet_id_list)):
        request_url = f"https://api.twitter.com/2/tweets?ids={tweet_id}&expansions=geo.place_id&place.fields=contained_within,country,country_code,full_name,geo,id,name,place_type"
        response = requests.get(request_url, headers=header).json()
        time.sleep(1)
        if 'includes' in response:
            #Geolocation available
            geo_data.append((label_list[i], response['includes']['places'][0]))
        else:
            continue
    return geo_data

In [150]:
tweet_ids = hate_tweets_pd['Tweet ID'].tolist()
labels = hate_tweets_pd['BERT_label'].tolist()

In [151]:
geo_data = get_location(tweet_ids, labels)

  0%|          | 0/17021 [00:00<?, ?it/s]

In [152]:
geo_data_extended.extend(geo_data)
#Count of hate speech tweets with geo data
len(geo_data_extended)

374

In [153]:
geo_data = geo_data_extended

In [213]:
geo_data[:5]

[(2,
  {'country_code': 'BW',
   'country': 'Botswana',
   'place_type': 'country',
   'name': 'Botswana',
   'full_name': 'Botswana',
   'geo': {'type': 'Feature',
    'bbox': [19.9976747, -26.9070352, 29.371142, -17.7782297],
    'properties': {}},
   'id': '8a927a7056322151'}),
 (2,
  {'country_code': 'US',
   'country': 'Etats-Unis',
   'place_type': 'city',
   'name': 'Washington',
   'full_name': 'Washington, DC',
   'geo': {'type': 'Feature',
    'bbox': [-77.119401, 38.801826, -76.909396, 38.9953797],
    'properties': {}},
   'id': '01fbe706f872cb32'}),
 (2,
  {'full_name': 'Sale, England',
   'name': 'Sale',
   'place_type': 'city',
   'id': '74790dfe0c08c72f',
   'country': 'Royaume-Uni',
   'country_code': 'GB',
   'geo': {'type': 'Feature',
    'bbox': [-2.3798084, 53.358611, -2.2737406, 53.4417876],
    'properties': {}}}),
 (2,
  {'place_type': 'city',
   'id': '8279f00eacbd9f3b',
   'full_name': 'Teaneck, NJ',
   'geo': {'type': 'Feature',
    'bbox': [-74.036896, 40.85

In [154]:
def parse_geo_hate(geo_hate_data):
    geo_data_list = []
    for _, geo_data in geo_hate_data:
        geo_data_list.append([geo_data['country'], geo_data['country_code']])
    return geo_data_list

In [155]:
country_code_hate_count = parse_geo_hate(geo_data)
column_names = ['country', 'country_code']
country_hate_pd = pd.DataFrame(country_code_hate_count, columns = column_names)
country_hate_pd

,country,country_code
0,Botswana,BW
1,Etats-Unis,US
2,Royaume-Uni,GB
3,Etats-Unis,US
4,Etats-Unis,US
...,...,...
369,Inde,IN
370,Etats-Unis,US
371,Etats-Unis,US
372,Etats-Unis,US


In [212]:
country_hate_count_pd = country_hate_pd.groupby(['country_code']).size().reset_index(name='counts')

In [157]:
import json

f = open('utils/iso3.json')
code_map = json.load(f)
f.close()

f = open('utils/names.json')
country_name_map = json.load(f)
f.close()

In [158]:
#Convert ISO-2 to ISO-3
country_hate_count_pd['country_name'] = country_hate_count_pd['country_code'].apply(lambda x: country_name_map[x])
country_hate_count_pd['country_code'] = country_hate_count_pd['country_code'].apply(lambda x: code_map[x])
country_hate_count_pd

,country_code,counts,country_name
0,ARE,1,United Arab Emirates
1,AUS,7,Australia
2,BEN,1,Benin
3,BRA,2,Brazil
4,BWA,1,Botswana
5,CAN,8,Canada
6,CHN,1,China
7,DEU,2,Germany
8,ESP,1,Spain
9,FRA,3,France


In [159]:
country_hate_count_pd.to_csv('data/country_wise_hate_count.csv')

In [160]:
import plotly.io as pio
pio.renderers.default = 'iframe'

In [199]:
import plotly.express as px
import plotly.graph_objects as go

fig = go.Figure(data=go.Choropleth(
    locations = country_hate_count_pd['country_code'],
    z = country_hate_count_pd['counts'],
    text = country_hate_count_pd['country_name'],
    # locationmode = 'country names',
    colorscale = 'Blues',
    autocolorscale=False,
    reversescale=False,
    marker_line_color='darkgray',
    marker_line_width=0.5,
    # colorbar_tickprefix = '$',
    colorbar_title = 'Hateful<br>Tweet Count',
))
fig.update_layout(
    title_text='Global Distribution of COVID-19 Anti-Asian Hate Speech from Twitter',
    geo=dict(
        showframe=False,
        showcoastlines=False,
        projection_type='equirectangular'
    ),
    annotations = [dict(
        x=0.55,
        y=-0.1,
        xref='paper',
        yref='paper',
        text='Source: <a href="https://arxiv.org/pdf/2005.12423.pdf">\
            Racism is a Virus: Anti-Asian Hate and Counterspeech in Social Media during the COVID-19 Crisis</a>',
        showarrow = False
    )]
)
# fig.write_image('image.png')
fig.show()

In [200]:
fig.write_image('images/world_hate_tweet.png')

### USA State-wise Distribution

In [201]:
from collections import defaultdict

def us_state_count(geo_data):
    state_wise_count = defaultdict(int)
    for _, data in geo_data:
        if data['country_code'] == 'US':
            state_wise_count[data['full_name'].split(', ')[1]] += 1
    return state_wise_count

In [211]:
us_geo_data = us_state_count(geo_data)
state_list = us_geo_data.keys()
state_count = us_geo_data.values()
us_geo_data

defaultdict(int,
            {'DC': 1,
             'NJ': 3,
             'MO': 2,
             'NY': 18,
             'OH': 7,
             'CA': 47,
             'NC': 4,
             'MD': 4,
             'FL': 14,
             'VA': 3,
             'PA': 9,
             'GA': 7,
             'TX': 13,
             'IN': 3,
             'NM': 1,
             'LA': 2,
             'USA': 40,
             'OK': 1,
             'TN': 5,
             'AZ': 4,
             'RI': 2,
             'IA': 2,
             'HI': 2,
             'MI': 2,
             'CT': 2,
             'CO': 1,
             'MN': 1,
             'IL': 4,
             'WA': 3,
             'MS': 1,
             'OR': 3,
             'MA': 7,
             'KY': 1,
             'NV': 3,
             'WV': 2,
             'UT': 1,
             'SC': 1,
             'AL': 1,
             'WI': 1,
             'NE': 1})

In [209]:
fig_state = px.choropleth(locations=state_list, locationmode="USA-states", color=state_count, scope="usa", 
                    color_continuous_scale="OrRd", title = 'USA Statewise Distribution of COVID-19 Anti-Asian Hate Speech from Twitter',
                         labels={'color': 'Hateful Tweet<br>Count'})
fig_state.show()

In [ ]:
fig_state.write_image('./images/usa_hate_tweet.png')